In [28]:
import numpy as np
import pandas as pd
import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape,  Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Adamax
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt


ImportError: ignored

In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
RATING_DATA_FILE_TRAIN = 'u1.base'
RATING_DATA_FILE_TEST = 'u1.test'
RATING_DATA_GEN_FILE = 'u_gen.data'
RATINGS_CSV_FILE_NORM = 'u_norm.data'
RATINGS_GEN_CSV_FILE = 'u_genr.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

In [0]:
rating_file_import_train = drive.CreateFile({'id':'1smKszlPQlT03Bbi7yLRIDIkd0c-XS-Y5'})
rating_file_import_train.GetContentFile(RATING_DATA_FILE_TRAIN)
rating_file_import_test = drive.CreateFile({'id':'1dxlfTQJiQ5MyewGyhk7Y4A9mETdnjknf'})
rating_file_import_test.GetContentFile(RATING_DATA_FILE_TEST)

In [5]:
ratings = pd.read_csv(RATING_DATA_FILE_TRAIN, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded')


80000 ratings loaded


In [13]:
test_ratings = pd.read_csv(RATING_DATA_FILE_TEST, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
test_ratings['user_emb_id'] = test_ratings['userid'] - 1
test_ratings['movie_emb_id'] = test_ratings['movieid'] - 1
print(str(len(test_ratings))+' ratings loaded')

20000 ratings loaded


In [0]:
def get_ncf_model(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input], output=prediction)
    print("ncf model")
    model.summary()

    return model

In [0]:
NCF_model
K_LATENT
hidden_dim
do

def set_ncf_model(parameter_hidden_dim, parameter_loss,parameter_optimizer,):
  K_LATENT = 20
  hidden_dim = parameter_hidden_dim
  do = 0.5
  NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do)
  NCF_model.compile(loss=parameter_loss,optimizer=parameter_optimizer,metrics=['mae'])

In [44]:

set_ncf_model(20,'mse','Adamax')
Users = ratings['user_emb_id'].values
Movies = ratings['movie_emb_id'].values
Ratings = ratings['rating'].values

ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________________________


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("pr..., inputs=[<tf.Tenso...)`


In [54]:
callbacks_ncf = [EarlyStopping('val_loss', patience=30), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=60, validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

Train on 56000 samples, validate on 24000 samples
Epoch 1/60
 3456/56000 [>.............................] - ETA: 2s - loss: 0.8052 - mean_absolute_error: 0.7043

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


56000/56000 [==============================] - 3s 53us/step - loss: 0.7986 - mean_absolute_error: 0.7034 - val_loss: 0.8901 - val_mean_absolute_error: 0.7658
Epoch 2/60
56000/56000 [==============================] - 3s 52us/step - loss: 0.8005 - mean_absolute_error: 0.7049 - val_loss: 0.8915 - val_mean_absolute_error: 0.7671
Epoch 3/60
56000/56000 [==============================] - 3s 52us/step - loss: 0.8017 - mean_absolute_error: 0.7050 - val_loss: 0.8953 - val_mean_absolute_error: 0.7684
Epoch 4/60
56000/56000 [==============================] - 3s 53us/step - loss: 0.7996 - mean_absolute_error: 0.7042 - val_loss: 0.8985 - val_mean_absolute_error: 0.7714
Epoch 5/60
56000/56000 [==============================] - 3s 53us/step - loss: 0.8003 - mean_absolute_error: 0.7049 - val_loss: 0.9030 - val_mean_absolute_error: 0.7750
Epoch 6/60
56000/56000 [==============================] - 3s 51us/step - loss: 0.7989 - mean_absolute_error: 0.7046 - val_loss: 0.9003 - val_mean_absolute_error: 0.77

KeyboardInterrupt: ignored

In [53]:

test_Users = test_ratings['user_emb_id'].values
test_Movies = test_ratings['movie_emb_id'].values
test_Ratings = test_ratings['rating'].values
predict=NCF_model.predict([test_Users,test_Movies])
df_predict=pd.DataFrame(data=predict,columns=['prediction'])
df_predict['ratings'] = test_Ratings

# print(df_predict)

main method - 0.7928911358368012
       prediction  ratings
0        3.846047        5
1        4.192070        3
2        4.560959        5
3        4.204177        5
4        3.296424        3
5        4.012165        4
6        4.472283        4
7        3.582150        3
8        2.974776        2
9        3.550973        3
10       3.534667        4
11       1.693192        2
12       3.341532        4
13       3.325302        5
14       3.932510        4
15       3.280835        3
16       3.633845        4
17       3.120405        3
18       3.284105        3
19       4.464338        4
20       4.391423        5
21       3.987499        4
22       3.166443        3
23       4.570510        5
24       3.905074        4
25       2.798226        3
26       3.968210        3
27       3.821422        3
28       3.019819        4
29       3.398594        3
...           ...      ...
19970    2.915899        3
19971    3.424267        3
19972    3.108876        4
19973    3.478194     

according to the first 

In [51]:
print('main method - '+str(sum(history_history_ncf.history['val_mean_absolute_error'])/len(history_history_ncf.history['val_mean_absolute_error'])))
set_ncf_model(16,'mse','Adamax')
callbacks_ncf = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=60, validation_split=.1, verbose=0, callbacks=callbacks_ncf, batch_size = 32)
predict=NCF_model.predict([test_Users,test_Movies])
df_predict=pd.DataFrame(data=predict,columns=['prediction'])
df_predict['ratings'] = test_Ratings
print('second method hidden layer size=16 - '+str(sum(history_history_ncf.history['val_mean_absolute_error'])/len(history_history_ncf.history['val_mean_absolute_error'])))
set_ncf_model(20,'mse','sgd')
callbacks_ncf = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=60, validation_split=.1, verbose=0, callbacks=callbacks_ncf, batch_size = 32)
predict=NCF_model.predict([test_Users,test_Movies])
df_predict=pd.DataFrame(data=predict,columns=['prediction'])
df_predict['ratings'] = test_Ratings
print('third method optimizer=SGD - '+str(sum(history_history_ncf.history['val_mean_absolute_error'])/len(history_history_ncf.history['val_mean_absolute_error'])))
set_ncf_model(20,'mape','Adamax')
callbacks_ncf = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history_history_ncf = NCF_model.fit([Users, Movies], Ratings, nb_epoch=60, validation_split=.1, verbose=0, callbacks=callbacks_ncf, batch_size = 32)
predict=NCF_model.predict([test_Users,test_Movies])
df_predict=pd.DataFrame(data=predict,columns=['prediction'])
df_predict['ratings'] = test_Ratings
print('fourth method loss function=mean absolute precentage error - '+str(sum(history_history_ncf.history['val_mean_absolute_error'])/len(history_history_ncf.history['val_mean_absolute_error'])))

main method - 0.8669673980308903
ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("pr..., inputs=[<tf.Tenso...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


second method hidden layer size=16 - 0.8759517161498467
ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
___________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


third method optimizer=SGD - 0.8805747488845479
ncf model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
___________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


fourth method loss function=mean absolute precentage error - 0.8833961687413128


The first method had shown the best results in based on the same parameters as the other method except the optimizer,loss function and the hidden dimentions.
From our research in the web we have concluded that the adamax optimizer is considered to be the best optimizer for this spessific problem. In addition the mse loss function has the closest aproximation to the real error value.As for the number of nodes in the hidden layer there isn't a good explanation for this spesific number, but through trial and error we deducted that 20 nodes show a decent result.
